# Astrophysics Chat Application 

In [ ]:
from ssec_tutorials import OLMO_MODEL

In [ ]:
from pathlib import Path
from qdrant_client import QdrantClient
from uuid import uuid4

In [ ]:
import panel as pn
import os
import textwrap
from langchain.llms import LlamaCpp
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.callbacks import CallbackManager
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
repo_root = Path("../../resources/data/").resolve()

In [ ]:
repo_root

In [ ]:
qdrant_path = repo_root / "qdrant/scipy_qdrant/"
qdrant_collection = "arxiv_astro-ph_abstracts_astropy_github_documentation"

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [ ]:
pn.extension()

In [ ]:
assert os.path.exists(qdrant_path)

In [ ]:
model_path = OLMO_MODEL

In [ ]:
@pn.cache
def get_vector_store():
    # If the Qdrant Vector Database Collection already exists, load it
    client = QdrantClient(path=str(qdrant_path))
    db = Qdrant(
        client=client,
        collection_name=qdrant_collection,
        embeddings=embedding
    )
    return db

In [ ]:
db = get_vector_store()

In [ ]:
def get_chain(callbacks):
    retriever = db.as_retriever(callbacks=callbacks, search_type="similarity", search_kwargs={"k": 2})
    
    # Callbacks support token-wise streaming
    callback_manager = CallbackManager(callbacks)
    olmo = LlamaCpp(
        model_path=str(model_path),
        callback_manager=callback_manager,
        temperature=0.8,
        n_ctx=4096,
        max_tokens=512,
        verbose=False,
        echo=False
    )
    prompt_template = PromptTemplate.from_template(
        template=olmo.client.metadata['tokenizer.chat_template'],
        template_format="jinja2",
        partial_variables={"add_generation_prompt": True, "eos_token": "<|endoftext|>"},
    )

    transformed_prompt_template = PromptTemplate.from_template(
        prompt_template.partial(
            messages=[
                {
                    "role": "user", 
                    "content": textwrap.dedent("""\
                    You are an astrophysics expert. Please answer the question on astrophysics based on the following context:
                    
                    {context}
                    
                    Question: {input}""")
                }
            ]
        ).format()
    )

    def format_docs(docs):
        text = "\n\n".join([d.page_content for d in docs])
        return text

    def hack(docs):
        # https://github.com/langchain-ai/langchain/issues/7290
        for callback in callbacks:
            callback.on_retriever_end(docs, run_id=uuid4())
        return docs

    return (
        {"context": retriever | hack | format_docs, "question": RunnablePassthrough()}
        | transformed_prompt_template
        | olmo
    )

In [ ]:
async def callback(contents, user, instance):
    callback_handler = pn.chat.langchain.PanelCallbackHandler(instance, user='OLMo', avatar='🌳')
    # Not return the result at the end of the generation
    # this prevents the model from repeating the result
    callback_handler.on_llm_end = lambda response, *args, **kwargs: None
    chain = get_chain(callbacks=[callback_handler])
    _ = await chain.ainvoke(contents)

In [ ]:
pn.chat.ChatInterface(callback=callback).servable()